### Imports utilizados

In [10]:
import requests
import time
import random
import re
import os
from tqdm import tqdm as tqdm
from bs4 import BeautifulSoup

### Parsing da Página Principal da Wikipedia

In [ ]:
a = requests.get('https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal')
soup = BeautifulSoup(a.text, 'html.parser')

Para cada link na seção "main" da página principal, verifica se o link representa um verbete, caso positivo, adiciona à um Set para evitar duplicatas

In [ ]:
links = soup.main.find_all('a')
all_verbets = set()
for link in links:
    if link.has_attr('href') and re.match(r'/wiki/.*', link['href']) and not re.match(r'/wiki/.*:.*', link['href']):
            all_verbets.add(link['href'])

112


Escolhe aleatoriamente um verbet da conjunto obtido da página principal, e faz o mesmo processamento. O loop é executado até que o conjunto de verbetes tenha 5000 verbetes.

In [ ]:
while(len(all_verbets) < 500):
    verbet = random.choice(list(all_verbets))
    a = requests.get('https://pt.wikipedia.org' + verbet)
    soup = BeautifulSoup(a.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        if link.has_attr('href') and re.match(r'/wiki/.*', link['href']) and not re.match(r'/wiki/.*:.*', link['href']):
            all_verbets.add(link['href'])
    time.sleep(random.randint(1,2))

{'/wiki/Matilda_Joslyn_Gage', '/wiki/Odontomachus', '/wiki/Mortes_em_2025', '/wiki/Golpe_de_Estado_na_Argentina_em_1976', '/wiki/Formiga', '/wiki/Carlos_Lacerda', '/wiki/%C3%93scar_Romero', '/wiki/Casuar', '/wiki/Guerra_Suja_na_Argentina', '/wiki/1947', '/wiki/Bebida_alco%C3%B3lica', '/wiki/Elei%C3%A7%C3%B5es_legislativas_portuguesas_de_2025', '/wiki/Massa_corporal', '/wiki/Estado_de_guerra', '/wiki/Dia_Internacional_do_Direito_%C3%A0_Verdade_sobre_as_Viola%C3%A7%C3%B5es_dos_Direitos_Humanos_e_pela_Dignidade_das_V%C3%ADtimas', '/wiki/Conte%C3%BAdo_livre', '/wiki/Tratado_de_paz', '/wiki/Aeroporto_de_Lukla', '/wiki/1917', '/wiki/Guerra_Israel-Hamas', '/wiki/Lu%C3%ADs_Montenegro', '/wiki/Eugene_Shoemaker', '/wiki/Padre_C%C3%ADcero', '/wiki/Tonelada', '/wiki/1826', '/wiki/Wlamir_Marques', '/wiki/Dia_Mundial_da_Tuberculose', '/wiki/Processo_de_Reorganiza%C3%A7%C3%A3o_Nacional', '/wiki/Esp%C3%A9cime', '/wiki/L%C3%ADngua_portuguesa', '/wiki/2016', '/wiki/Filiz_Ak%C4%B1n', '/wiki/Fic%C3%A7%C3%

Cria o diretório para armazenar os verbetes, percorre todo o conjunto e salva o HTML

In [12]:
os.makedirs('verbetes', exist_ok=True)
for verbet in tqdm(all_verbets):
    titulo = verbet.split('/')[-1]
    soup = BeautifulSoup(a.text, 'html.parser')
    with open(f'verbetes/{titulo}.html', 'w') as f:
        f.write(soup.prettify())
    time.sleep(random.randint(1,2))

  1%|▏         | 12/820 [00:21<23:54,  1.78s/it]


KeyboardInterrupt: 